# Ejercicio
* En finanzas matemáticas, la ecuación de Black-Scholes es una ecuación diferencial parcial (PDE) que rige la evolución del precio de una opción europea de compra o venta europea según el modelo de [Black-Scholes](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_equation)

* Una fórmula de negociación de opciones financieras utilizada para el precio de inversión estimados
* La fórmula calcula el precio de las opciones europeas 'put' y 'call'

###  Version bucle
* La función *black_scholes* se implementa con un bucle que calcula las opciones 'put' y 'call'
* Sustituir dicha tipología en forma de bucle por llamadas a la librería **NumPy** para acelerar dicha función

In [7]:
import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

from math import log, sqrt, exp, erf
import numpy as np
invsqrt = lambda x: 1.0/sqrt(x)

def black_scholes ( nopt, price, strike, t, rate, vol, call, put ):
    mr = -rate
    sig_sig_two = vol * vol * 2

    for i in range(nopt):
        P = float( price [i] )
        S = strike [i]
        T = t [i]

        a = log(P / S)
        b = T * mr

        z = T * sig_sig_two
        c = 0.25 * z
        y = invsqrt(z)

        w1 = (a - b + c) * y
        w2 = (a - b - c) * y

        d1 = 0.5 + 0.5 * erf(w1)
        d2 = 0.5 + 0.5 * erf(w2)

        Se = exp(b) * S

        call [i] = P * d1 - Se * d2
        put [i] = call [i] - P + Se


        


In [9]:
import time

timing = {}

nopt=1_000_000

price, strike, t = gen_data(nopt)
call = [0.0 for i in range(nopt)]
put = [-1.0 for i in range(nopt)]
price=list(price)
strike=list(strike)
t=list(t)


t1=time.time()
black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)
t2 = time.time()
print("With for loop and appending it took {} seconds".format(t2-t1))
timing['loop'] = (t2-t1)


With for loop and appending it took 2.6318583488464355 seconds


In [57]:
def my_erf(z):
    t = 1.0 / (1.0 + 0.5 * np.abs(z))
    # use Horner's method
    ans = 1 - t * np.exp( -z*z -  1.26551223 +
                            t * ( 1.00002368 +
                            t * ( 0.37409196 + 
                            t * ( 0.09678418 + 
                            t * (-0.18628806 + 
                            t * ( 0.27886807 + 
                            t * (-1.13520398 + 
                            t * ( 1.48851587 + 
                            t * (-0.82215223 + 
                            t * ( 0.17087277))))))))))
    if z >= 0.0:
        return ans
    else:
        return -ans

def black_scholes_numpy ( nopt, price, strike, t, rate, vol, call, put ):
    mr = -rate
    sig_sig_two = vol * vol * 2

    P = price
    S = strike
    T = t

    a = np.log(P / S)
    b = T * mr

    z = T * sig_sig_two
    c = 0.25 * z
    y = np.divide(1, np.sqrt(z))

    w1 = np.multiply(a - b + c,  y)
    w2 = np.multiply(a - b - c, y)

    d1 = 0.5 + 0.5 * my_erf(w1)
    d2 = 0.5 + 0.5 * my_erf(w2)

    Se = np.exp(b) * S

    call[:] = P * d1 - Se * d2
    put[:] = call - P + Se

In [58]:
t1=time.time()
black_scholes_numpy(nopt, np.array(price), np.array(strike), np.array(t), RISK_FREE, VOLATILITY, call, put)
t2 = time.time()
print("With for Numpy it took {} seconds".format(t2-t1))
timing['loop'] = (t2-t1)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [54]:
erf(2)


0.9953222650189527

In [21]:
np.array(t)

array([1.83958489, 1.54873933, 1.57687454, ..., 1.97530414, 1.82209422,
       1.83881122])